In [ ]:
import PyPDF2
import io
from PIL import Image
from IPython.display import display

from pdf2image import convert_from_path

def convert_pdf_to_images(pdf_path):
    images = convert_from_path(pdf_path)
    for i, img in enumerate(images):
        display(img)   # Display the image
        img.save(f'output_page_{i}.jpg', 'JPEG')


def extract_images_from_pdf(pdf_path):
    input_pdf = PyPDF2.PdfFileReader(open(pdf_path, "rb"))

    for page_num in range(input_pdf.getNumPages()):
        page = input_pdf.getPage(page_num)
        
        if '/Resources' in page and '/XObject' in page['/Resources']:
            xobjects = page['/Resources']['/XObject'].getObject()
            for obj in xobjects:
                xobj = xobjects[obj]
                if xobj['/Subtype'] == '/Image':
                    try:
                        img_data = xobj.getData()
                        
                        # Analyze the first few bytes of the image data
                        print(f"Image {obj} on page {page_num} first 10 bytes: {img_data[:10]}")
                        
                        img = Image.open(io.BytesIO(img_data))
                        display(img)  # Display the image within the notebook
                        print(f"Image {obj} on page {page_num} extracted successfully.")
                    except Exception as e:
                        print(f"Failed to extract image {obj} on page {page_num}: {e}")


In [29]:
!pip install img2pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for img2pdf: filename=img2pdf-0.5.1-py3-none-any.whl size=49277 sha256=b533abcd3c0dcd03e920bb8dfd1bf097e5f3b067a9d5d205aa4e0b0dbd570d51
  Stored in directory: /Users/ankit/Library/Caches/pip/wheels/31/82/cb/7acc5b35f17889ff58f207f8f4379d34029fa36473411d0eb7
Successfully built img2pdf


In [32]:
import fitz  # PyMuPDF
from PIL import Image
from io import BytesIO
import PyPDF2
from pdf2image import convert_from_path
import os
import img2pdf

def compress_pdf(input_pdf_path, output_pdf_path, max_size=5 * 1024 * 1024):
    images = convert_from_path(input_pdf_path, first_page=0, last_page=1)

    # The first page is now stored as an image in `images[0]`
    image = images[0]

    # Step 2: Check the size of the image and resize if necessary
    output_image_path = "first_page_image.jpg"
    image.save(output_image_path, "JPEG")

    # Get the size of the image
    image_size = os.path.getsize(output_image_path)

    # Resize the image if it exceeds 5 MB
    max_size = 5 * 1024 * 1024  # 5 MB
    if image_size > max_size:
        # Reduce the image size while keeping the aspect ratio
        image.thumbnail((image.width // 2, image.height // 2))
        image.save(output_image_path, "JPEG")

    # Step 3: Convert the image back to a PDF
    with open(output_pdf_path, "wb") as f:
        f.write(img2pdf.convert(output_image_path))

    # Clean up the temporary image file
    os.remove(output_image_path)

    print(f"First page saved as {output_pdf_path}")

SyntaxError: invalid syntax (3473184293.py, line 1)

In [15]:
import os 
def compress_pdf(input_pdf_path, output_pdf_path):
    os.system(f'gs -sDEVICE=pdfwrite -dCompatibilityLevel=1.4 -dPDFSETTINGS=/screen -dNOPAUSE -dQUIET -dBATCH -sOutputFile={output_pdf_path} {input_pdf_path}')


In [2]:
def compress_pdf(input_pdf_path, output_pdf_path):
    os.system(f'qpdf --linearize {input_pdf_path} {output_pdf_path}')

In [31]:
compress_pdf('/Users/ankit/code/backend/media/attachment/1685007607.pdf', 'test_fitz.pdf')

First page saved as test_fitz.pdf


In [ ]:
convert_pdf_to_images('/Users/ankit/code/backend/media/receipt_draft/2024/August/12/b23ed1a4-b56a-483c-9cfe-df56f079c2f9/97.pdf')